# Cards

In [ ]:
class Card():
    
        def __init__(self, suit, value):
            self.suit = suit
            self.value = value
    
        def __repr__(self):
            return "{} of {}".format(self.value, self.suit)
class Deck():

    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    values = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight',
                'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
    
    def __init__(self):
        self.deck = []
        for suit in Deck.suits:
            for value in Deck.values:
                self.deck.append(Card(suit, value))